In [1]:
!nvidia-smi
!pip install missingpy

zsh:1: command not found: nvidia-smi


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

In [5]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [6]:
import sys
import numpy as np
import pandas as pd

from missingpy import MissForest
from dsan import Imputer as DSANImputer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [7]:
seed = 128
set_seed(seed)

In [8]:
def get_imputer(config):
    
    model = config['model']
    
    if model == 'DSAN':
        imputer = DSANImputer(rep_dim=config['rep_dim'],
                num_heads=config['num_heads'],
                n_hidden=config['n_hidden'],
                lr=config['lr'],
                weight_decay=config['weight_decay'],
                batch_size=config['batch_size'],
                epochs=config['epochs'],
                noise_percent=config['noise_percent'],
                stopped_epoch=config['stopped_epoch'])
    
    elif model == 'MissForest':
        imputer = MissForest(random_state=0)
        
    elif model == 'Statistics':
        imputer = None
        
    return imputer

In [11]:
def experiment_run(data_name, config, test=False):
    # Retrieve dataset
    dataset = get_data(data_name)
    print(f"Retrieved dataset for {data_name}")

    # Limit to 1000 samples if test flag is true
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])
        print("Limited dataset to 1000 samples for testing")

    k = 5
    percent_missing_lst = [p for p in range(5, 25, 5)]
    X_origin, n_col, num_vars, cat_vars = dataset

   # Apply category mapping
    X_origin = category_mapping(X_origin, cat_vars)
    print("Applied category mapping")

    # Check if X_origin is modified in-place or replaced
    if X_origin is not None:
        print("X_origin is replaced with a new version where categorical variables have been mapped to numerical values.")
    else:
        print("X_origin is modified in-place, with categorical variables mapped to numerical values.")

    # Print data size
    print("Data Size: {}".format(X_origin.shape))


    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation setup
    n_valid = len(idx_lst) // k
    print(f"Starting k-fold validation with k={k}")

    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        print(f"\nProcessing {percent_missing}% missing data")

        for it, n in enumerate(range(0, n_sample, n_valid)):
            print(f"\nFold {it + 1}")
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                print("Insufficient data for validation, breaking loop")
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :]

            # Generating incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan
            print("Generated incomplete data with missing values for fold {}:".format(it+1))
            print("Original data with missing values (subset):")
            print(X_incomplete[:5])  # Adjust the number of rows as needed

            # Imputation
            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            print("Completed imputation for fold {}:".format(it+1))

            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            print("X_res with imputed values (subset):")
            print(X_res[:5])  # Adjust the number of rows as needed

            # Calculate metrics
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_regress(data_name=data_name, train_array=X_res, test_array=X_test, num_vars=num_vars, cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print("Metrics for this fold:", metric)

        # Average results
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(f"Average {key} for {percent_missing}% missing data: {value}")

        results[percent_missing] = avg_result

    print("Completed all iterations")
    return results


In [12]:
config = {
    'model': 'DSAN',
    'rep_dim': 32,
    'num_heads': 16,
    'n_hidden': 2,
    'lr': 3e-3,
    'weight_decay': 1e-5,
    'batch_size': 32,
    'epochs': 34,
    'noise_percent': 15,
    'stopped_epoch': 15
}

In [13]:
data_name_lst = ['insurance']

result_dir = './results1'
os.makedirs(result_dir)

for data_name in data_name_lst:
    results = experiment_run(data_name, config, test=True)
    results['config'] = config
    results['dataset'] = data_name
    result_path = os.path.join(result_dir, '{}.json'.format(data_name))
    write_json(results, result_path)

Retrieved dataset for insurance
Limited dataset to 1000 samples for testing
Applied category mapping
X_origin is replaced with a new version where categorical variables have been mapped to numerical values.
Data Size: (1000, 7)
Starting k-fold validation with k=5

Processing 5% missing data

Fold 1
Generated incomplete data with missing values for fold 1:
Original data with missing values (subset):
[[28 1 33.0 3 1 nan nan]
 [32 1 28.88 nan 1 2 3866.8552]
 [31 0 25.74 0 1 1 3756.6216]
 [46 0 33.44 1 1 nan 8240.5896]
 [37 1 29.83 2 1 3 6406.4107]]


100%|███████████████████████████████████████████| 34/34 [00:23<00:00,  1.43it/s]


Completed imputation for fold 1:
X_res with imputed values (subset):
[[28 1 33.0 3 1 3 8232.080078125]
 [32 1 28.88 1.4683914184570312 1 2 3866.8552]
 [31 0 25.74 0 1 1 3756.6216]
 [46 0 33.44 1 1 3 8240.5896]
 [37 1 29.83 2 1 3 6406.4107]]
Metrics for this fold: {'nrmse': 0.3271, 'col_1_error_rate': 0.5263, 'col_4_error_rate': 0.0455, 'col_5_error_rate': 0.8462, 'total_error_rate': 0.4545, 'clf_aucroc': 37805139.9444}

Fold 2
Generated incomplete data with missing values for fold 2:
Original data with missing values (subset):
[[19 0 27.9 0 nan 0 16884.924]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 1 1 4449.462]
 [33 1 22.705 0 1 2 21984.47061]
 [32 1 28.88 0 1 2 3866.8552]]


100%|███████████████████████████████████████████| 34/34 [00:21<00:00,  1.59it/s]


Completed imputation for fold 2:
X_res with imputed values (subset):
[[19 0 27.9 0 0 0 16884.924]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 1 1 4449.462]
 [33 1 22.705 0 1 2 21984.47061]
 [32 1 28.88 0 1 2 3866.8552]]
Metrics for this fold: {'nrmse': 0.3888, 'col_1_error_rate': 0.4444, 'col_4_error_rate': 0.1026, 'col_5_error_rate': 0.617, 'total_error_rate': 0.4046, 'clf_aucroc': 42461949.5522}

Fold 3
Generated incomplete data with missing values for fold 3:
Original data with missing values (subset):
[[19 0 27.9 nan 0 0 16884.924]
 [nan 1 33.77 1 nan 1 1725.5523]
 [28 nan 33.0 3 1 1 4449.462]
 [33 1 22.705 0 1 2 21984.47061]
 [32 1 28.88 0 1 2 3866.8552]]


...

Completed imputation for fold 3:
X_res with imputed values (subset):
[[19 0 27.9 0.8390485048294067 0 0 16884.924]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 1 1 4449.462]
 [33 1 22.705 0 1 2 5285.45263671875]
 [32.931114196777344 1 28.88 1.6381959915161133 1 2 3866.8552]]
Metrics for this fold: {'nrmse': 0.4813, 'col_1_error_rate': 0.426, 'col_4_error_rate': 0.1128, 'col_5_error_rate': 0.7416, 'total_error_rate': 0.4562, 'clf_aucroc': 39185847.3237}

Fold 4
Generated incomplete data with missing values for fold 4:
Original data with missing values (subset):
[[19 nan nan 0 0 0 nan]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 nan 1 4449.462]
 [33 1 nan 0 1 nan 21984.47061]
 [32 1 28.88 0 1 2 3866.8552]]


100%|███████████████████████████████████████████| 34/34 [00:28<00:00,  1.18it/s]


Completed imputation for fold 4:
X_res with imputed values (subset):
[[19 1 30.40291404724121 0 0 0 20440.734375]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 1 1 4449.462]
 [33 1 30.450634002685547 0 1 1 21984.47061]
 [32 1 28.88 0 1 2 3866.8552]]
Metrics for this fold: {'nrmse': 0.5377, 'col_1_error_rate': 0.481, 'col_4_error_rate': 0.1304, 'col_5_error_rate': 0.7467, 'total_error_rate': 0.4456, 'clf_aucroc': 31200505.9189}

Fold 5
Generated incomplete data with missing values for fold 5:
Original data with missing values (subset):
[[19 0 27.9 0 0 nan 16884.924]
 [18 1 33.77 1 1 1 1725.5523]
 [33 1 22.705 0 1 2 21984.47061]
 [31 nan nan 0 nan 1 3756.6216]
 [46 0 nan 1 1 nan 8240.5896]]


100%|███████████████████████████████████████████| 34/34 [00:26<00:00,  1.30it/s]


Completed imputation for fold 5:
X_res with imputed values (subset):
[[19 0 27.9 0 0 1 16884.924]
 [18 1 33.77 1 1 1 1725.5523]
 [33 1 22.705 0 1 2 21984.47061]
 [31 0 31.51054573059082 0 1 1 3756.6216]
 [46 0 30.23199462890625 1 1 1 8240.5896]]
Metrics for this fold: {'nrmse': 0.4253, 'col_1_error_rate': 0.4356, 'col_4_error_rate': 0.1528, 'col_5_error_rate': 0.6667, 'total_error_rate': 0.4207, 'clf_aucroc': 31389789.2269}
Average nrmse for 20% missing data: 0.49989999999999996
Average col_1_error_rate for 20% missing data: 0.45196
Average col_4_error_rate for 20% missing data: 0.12808000000000003
Average col_5_error_rate for 20% missing data: 0.7063200000000001
Average total_error_rate for 20% missing data: 0.43644
Average clf_aucroc for 20% missing data: 36403552.71570001
Completed all iterations


In [15]:
config = {
    'model': 'MissForest'
    }

In [16]:
experiment_run('insurance', config, test=True)

Retrieved dataset for insurance
Limited dataset to 1000 samples for testing
Applied category mapping
X_origin is replaced with a new version where categorical variables have been mapped to numerical values.
Data Size: (1000, 7)
Starting k-fold validation with k=5

Processing 5% missing data

Fold 1
Generated incomplete data with missing values for fold 1:
Original data with missing values (subset):
[[19 0 27.9 0 0 0 nan]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 nan 1 4449.462]
 [31 0 25.74 0 1 1 3756.6216]
 [46 0 33.44 1 1 1 8240.5896]]
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Completed imputation for fold 1:
X_res with imputed values (subset):
[[19 0 27.9 0 0 0 17107.402605999992]
 [18 1 33.77 1 1 1 1725.5523]
 [28 1 33.0 3 1.0 1 4449.462]
 [31 0 25.74 0 1 1 3756.6216]
 [46 0 33.44 1 1 1 8240.5896]]
Metrics for this fold: {'nrmse': 0.4377, 'col_1_error_rate': 0.4865, 'col_4_error_rate': 0.1071, 'col_5_error_rate': 0.5263, 'total_error_rate': 0.3981, 'clf_aucroc': 4026816

{5: {'nrmse': 0.36196,
  'col_1_error_rate': 0.42517999999999995,
  'col_4_error_rate': 0.06649999999999999,
  'col_5_error_rate': 0.68566,
  'total_error_rate': 0.38612,
  'clf_aucroc': 35610693.37522},
 10: {'nrmse': 0.41546000000000005,
  'col_1_error_rate': 0.4549,
  'col_4_error_rate': 0.07698,
  'col_5_error_rate': 0.6965,
  'total_error_rate': 0.39996,
  'clf_aucroc': 35609725.57057999},
 15: {'nrmse': 0.45152000000000003,
  'col_1_error_rate': 0.44764,
  'col_4_error_rate': 0.061380000000000004,
  'col_5_error_rate': 0.6248400000000001,
  'total_error_rate': 0.38106,
  'clf_aucroc': 35335115.39186},
 20: {'nrmse': 0.48793999999999993,
  'col_1_error_rate': 0.46747999999999995,
  'col_4_error_rate': 0.10154,
  'col_5_error_rate': 0.65954,
  'total_error_rate': 0.4123799999999999,
  'clf_aucroc': 35674720.311079994}}